In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
from mmctools.wrf import preprocessing as wps
import os
import glob
import numpy as np

# Notes on the SetupWRF class:
 - Any namelist variable that is max_dom can be specified as a single value to be defined the same across all domains, or as a list that is the same length as the number of domains
 - If you're trying to add a namelist variable that isn't working, it likely isn't implemented yet... contact hawbecke(at)ucar.edu for assisstance
 - Downloading IC/BC data is not recommended within a Jupyter Notebook
 - This will not run the simulations for you (yet) but does create submission scripts for Cheyenne. If you are on a different system, contact hawbecke(at)ucar.edu for assisstance in adding your system to the possible submission script templates

# User Settings

In [3]:
# Setup WRF directory here:
main_directory    = '/glade/scratch/hawbecke/WRF/MMC/Downscaling/'
# Location of WRF run directory:
wrf_exe_location = '/glade/work/hawbecke/Models/WRF/WRFvMMC/WRF/run/'
# Location of WPS executables
wps_exe_location = '/glade/work/hawbecke/Models/WRF/WRFvMMC/WPS/'

# Which ICBC type
icbc_type = 'ERAI'  # ERAI, ERA5, FNL
# Where to download/link IC/BC data:
icbc_directory = '/glade/scratch/hawbecke/WRF/ICBC/'

# Case specifics - This is an example of 60 hour runs for 6 days, 2 day intervals (spinup 12 h)
case_start  = '2016-03-07 00:00:00'
case_end    = '2016-03-13 00:00:00'
# --Time between runs
case_delta  = '2d'
# --Runtime
case_length = '60h'

# Submission script info
submission_dict = {
            'account_key' : 'nwsa0002', # account key
         'walltime_hours' : {'wrf':6,'wps':2,'real':1}, # number of hours to allocate to each program
             'user_email' : 'hawbecke@ucar.edu', # email for status reporting
                  'nodes' : 2, # Number of nodes for real and wrf (wps=1 by default)
                   }

setup_dict = {
             'start_date' : '2020-04-03 18:00:00', # This will be overwritten if in a loop
               'end_date' : '2020-04-05 06:00:00', # This will be overwritten if in a loop
              'icbc_type' : icbc_type,
      'number_of_domains' : 2,
                    'dxy' : 6250,
              'time_step' : 30,
           'max_ts_level' : 51,
         'num_eta_levels' : 88,
             'eta_levels' : '1.00000,  0.99935,  0.99871,  0.99806,\n   0.99742,  0.99677,  0.99609,  0.99538,\n   0.99464,  0.99386,  0.99304,  0.99218,\n   0.99127,  0.99032,  0.98933,  0.98829,\n   0.98719,  0.98605,  0.98484,  0.98358,\n   0.98226,  0.98087,  0.97941,  0.97789,\n   0.97629,  0.97461,  0.97285,  0.97101,\n   0.96908,  0.96705,  0.96493,  0.96271,\n   0.96038,  0.95793,  0.95538,  0.95270,\n   0.94989,  0.94696,  0.94388,  0.94066,\n   0.93729,  0.93360,  0.92955,  0.92512,\n   0.92026,  0.91495,  0.90914,  0.90278,\n   0.89584,  0.88825,  0.87997,  0.87095,\n   0.86112,  0.85042,  0.83879,  0.82617,\n   0.81247,  0.79764,  0.78161,  0.76430,\n   0.74566,  0.72562,  0.70412,  0.68112,\n   0.65658,  0.63048,  0.60281,  0.57359,\n   0.54285,  0.51066,  0.47711,  0.44234,\n   0.40652,  0.36985,  0.33259,  0.29501,\n   0.25746,  0.22028,  0.18491,  0.15347,\n   0.12553,  0.10069,  0.07861,  0.05898,\n   0.04154,  0.02603,  0.01225,  0.00000',
      'parent_grid_ratio' : [1,5],
      'parent_time_ratio' : [1,5],
                 'istart' : [1, 200],
                 'jstart' : [1, 200],
                     'nx' : [480, 481],
                     'ny' : [480, 481],
                'ref_lat' : 39.8,
                'ref_lon' : -73.0,
              'true_lat1' : 25.0,
              'true_lat2' : 55.0,
              'stand_lon' : -70.0,
           'geogrid_args' : '30s+topobath_30s',
         'geog_data_path' : '/glade/work/hawbecke/geog',
       'history_interval' : [120,60],
       'auxinput4_inname' : '"wrflowinp_d<domain>"',
     'auxinput4_interval' : 360,
      'io_form_auxinput4' : 2,
      #'iofields_filename' : '"myoutfields.txt"',      
                  'debug' : 0,
        'p_top_requested' : 10000,
               'feedback' : 0,
          'smooth_option' : 0,
             'mp_physics' : 5,
      'sf_sfclay_physics' : [5,5],
     'sf_surface_physics' : [2,2],
         'bl_pbl_physics' : [5,5],
             'cu_physics' : [6,0],
             'hybrid_opt' : 0,
            'use_theta_m' : 0,
          }


# tslist settings (optional):
twr_lat = [ 39.969278,  39.546772,  39.2717,  41.325567]
twr_lon = [-72.716692, -73.428892, -73.8892, -70.568883]
twr_names = ['E05','E06','Atlantic Shores','DOE_MV']
twr_abbreviation = ['E05','E06','ATS','DMV']

In [4]:
case_dates = pd.date_range(case_start,case_end,freq=case_delta,closed='left')
cases = []
for cc,case in enumerate(case_dates):
    case_str = str(case).split(' ')[0].replace('-','')
    case_str = '{}_{}'.format(icbc_type,case_str)
    cases.append(case_str)
    run_directory = '{}{}/'.format(main_directory,case_str)
    setup_dict['start_date'] = str(case)
    setup_dict['end_date'] = str(case + pd.to_timedelta(case_length))
    wrf_setup = wps.SetupWRF(run_directory=run_directory,
                              icbc_directory=icbc_directory,
                              executables_dict={'wrf':wrf_exe_location,'wps':wps_exe_location},
                              setup_dict=setup_dict)
    wrf_setup.link_executables()
    wrf_setup.write_wps_namelist()
    wrf_setup.write_namelist_input()
    wrf_setup.write_submission_scripts(submission_dict=submission_dict,hpc='cheyenne')
    wrf_setup.create_tslist_file(lat=twr_lat,lon=twr_lon,twr_names=twr_names,twr_abbr=twr_abbreviation)
    # Not recommended in a Jupyter Notebook:
    ###wrf_setup.get_icbcs()

wrf_setup.create_submitAll_scripts(main_directory,cases,['wps','real','wrf'])


# If you want to edit IO:

In [5]:
vars_to_remove_LES = ['EDMF_A','EDMF_QC','EDMF_QT','EDMF_ENT','EDMF_W','EDMF_THL','QVAPOR','SH20',
                   'F_ICE_PHY','F_RAIN_PHY','F_RIMEF_PHY','QCLOUD','QRAIN','QSNOW','QHAIL','QGRAUP','QVAPOR',
                  'SMCREL','SHDMAX','SHDMIN','SNOALB','SEAICE','XICEM','SFROFF',
                  'UDROFF','IVGTYP','ACSNOM','SNOW','SNOWNC','SNOWH','CANWAT','COSZEN','LAI','TSLB',
                  'GLW','TMN','SNOWC','SR','QICE','QNICE','QNSNOW','QNRAIN','QNGRAUPEL',
                  'ACLWDNB','ACLWDNBC','ACLWDNT','ACLWDNTC','ACLWUPB','ACLWUPBC','ACLWUPT','ACLWUPTC',
                  'ACSWDNB','ACSWDNBC','ACSWDNT','ACSWDNTC','ACSWUPB','ACSWUPBC','ACSWUPT','ACSWUPTC',
                  'AFWA_RAIN','ALBBCK','ALBEDO','E','EMISS','NEST_POS','LAKEMASK',
                  'MAPFAC_M','MAPFAC_U','MAPFAC_V','MAPFAC_MX','MAPFAC_MY','MAPFAC_UX','MAPFAC_UY','MAPFAC_VX','MF_VX_INV','MAPFAC_VY','SINALPHA',
                  'I_ACLWDNB','I_ACLWDNBC','I_ACLWDNT','I_ACLWDNTC','I_ACLWUPB','I_ACLWUPBC','I_ACLWUPT','I_ACLWUPTC',
                  'I_ACSWDNB','I_ACSWDNBC','I_ACSWDNT','I_ACSWDNTC','I_ACSWUPB','I_ACSWUPBC','I_ACSWUPT','I_ACSWUPTC',
                  'GRAUPELNC','HAILNC','NOAHRES','NUPDRAFT','OLR','XICEM','ZS','DZS',
                  'VAR_SSO','RDX','RDY','SMOIS','VAR','ALBBCK','XLAND','EL_PBL',
                  'P_HYD','CWM','RQCBLTEN','RQIBLTEN','RQNIBLTEN','RQVBLTEN','RUBLTEN','RVBLTEN','RTHBLTEN',
                  ]

vars_to_remove_MESO = vars_to_remove_LES.copy()
vars_to_remove_MESO.extend(['RU_TEND','RU_TEND_ADV','RU_TEND_PGF','RU_TEND_COR','RU_TEND_CURV','RU_TEND_HDIFF','RU_TEND_PHYS','T_TEND_ADV',
                            'RV_TEND','RV_TEND_ADV','RV_TEND_PGF','RV_TEND_COR','RV_TEND_CURV','RV_TEND_HDIFF','RV_TEND_PHYS',
                            'U_G','V_G','U_G_TEND','V_G_TEND','MUU','MUV','MUT'])


vars_to_add_LES = ['ZNT','SST','SSTSK','SST_INPUT','UST','QFX','HFX','LH','ACHFX','ACLHF','GRDFLX','ACGRDFLX','SMOIS','PBLH',
                   'SWUPB','SWUPBC','SWDNB','SWDNBC','LWUPB','LWUPBC','LWDNB','LWDNBC',
                   'RU_TEND','RU_TEND_ADV','RU_TEND_PGF','RU_TEND_COR','RU_TEND_CURV','RU_TEND_HDIFF','RU_TEND_PHYS','T_TEND_ADV',
                   'RV_TEND','RV_TEND_ADV','RV_TEND_PGF','RV_TEND_COR','RV_TEND_CURV','RV_TEND_HDIFF','RV_TEND_PHYS',
                   'U_G','V_G','U_G_TEND','V_G_TEND','MUU','MUV','MUT']

vars_to_add_MESO = ['ZNT','SST','SSTSK','SST_INPUT','UST','QFX','HFX','LH','ACHFX','ACLHF','GRDFLX','ACGRDFLX','SMOIS','PBLH',
                     'SWUPB','SWUPBC','SWDNB','SWDNBC','LWUPB','LWUPBC','LWDNB','LWDNBC','RAINSH','RAINNC','RAINC',
                     'TKE','M11','M22','M33','M12','M13','M23','MH1','MH2','MH3','MQ1','MQ2','MQ3']
    

### Add the line for iofields_filename to your setup dictionary:

In [6]:
setup_dict['iofields_filename'] = ['"myoutfields_MESO.txt"','"myoutfields_LES.txt"']

In [7]:
case_dates = pd.date_range(case_start,case_end,freq=case_delta,closed='left')
cases = []
for cc,case in enumerate(case_dates):
    case_str = str(case).split(' ')[0].replace('-','')
    case_str = '{}_{}'.format(icbc_type,case_str)
    cases.append(case_str)
    run_directory = '{}{}/'.format(main_directory,case_str)
    setup_dict['start_date'] = str(case)
    setup_dict['end_date'] = str(case + pd.to_timedelta(case_length))
    wrf_setup = wps.SetupWRF(run_directory=run_directory,
                              icbc_directory=icbc_directory,
                              executables_dict={'wrf':wrf_exe_location,'wps':wps_exe_location},
                              setup_dict=setup_dict)
    # Need to update your namelist:
    wrf_setup.write_namelist_input()
    # Make the IO files:
    wrf_setup.write_io_fieldnames(vars_to_remove=[vars_to_remove_MESO,vars_to_remove_LES],vars_to_add=[vars_to_add_MESO,vars_to_add_LES])
